In [17]:
import numpy as np
import pandas as pd     
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns    
from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv("latimes-county-totals.csv")

In [19]:
df.head()

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
0,2020-01-26,Alameda,1,0,0,NaN,NaN
1,2020-01-31,Alameda,1,0,0,0.0,0.0
2,2020-02-02,Alameda,1,0,0,0.0,0.0
3,2020-02-20,Alameda,1,0,0,0.0,0.0
4,2020-02-21,Alameda,1,0,0,0.0,0.0


In [20]:
df.describe()

,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
count,2485.000000,2485.000000,2485.000000,2432.000000,2432.000000
mean,59.756539,229.698189,6.879276,15.485197,0.591283
std,32.786933,974.019813,37.795395,70.478349,3.502466
min,1.000000,0.000000,0.000000,-2.000000,0.000000
25%,31.000000,2.000000,0.000000,0.000000,0.000000
50%,65.000000,14.000000,0.000000,1.000000,0.000000
75%,85.000000,101.000000,2.000000,7.000000,0.000000
max,115.000000,16449.000000,732.000000,1474.000000,76.000000


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 0 to 2484
Data columns (total 7 columns):
date                   2485 non-null object
county                 2485 non-null object
fips                   2485 non-null int64
confirmed_cases        2485 non-null int64
deaths                 2485 non-null int64
new_confirmed_cases    2432 non-null float64
new_deaths             2432 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 136.0+ KB


In [22]:
df_la = df[df['county'] == 'Los Angeles']

In [23]:
print(df_la)

           date       county  fips  confirmed_cases  deaths  \
657  2020-01-26  Los Angeles    37                1       0   
658  2020-01-27  Los Angeles    37                1       0   
659  2020-01-28  Los Angeles    37                1       0   
660  2020-01-29  Los Angeles    37                1       0   
661  2020-01-30  Los Angeles    37                1       0   
..          ...          ...   ...              ...     ...   
740  2020-04-18  Los Angeles    37            12051     577   
741  2020-04-19  Los Angeles    37            12349     601   
742  2020-04-20  Los Angeles    37            13823     619   
743  2020-04-21  Los Angeles    37            15165     666   
744  2020-04-22  Los Angeles    37            16449     732   

     new_confirmed_cases  new_deaths  
657                  NaN         NaN  
658                  0.0         0.0  
659                  0.0         0.0  
660                  0.0         0.0  
661                  0.0         0.0  
..       

In [24]:
df_la.columns

Index(['date', 'county', 'fips', 'confirmed_cases', 'deaths',
       'new_confirmed_cases', 'new_deaths'],
      dtype='object')

In [25]:
df_la = df_la.drop(labels=['date','county','fips'], axis=1)
print (df_la.info())
df_la.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 657 to 744
Data columns (total 4 columns):
confirmed_cases        88 non-null int64
deaths                 88 non-null int64
new_confirmed_cases    87 non-null float64
new_deaths             87 non-null float64
dtypes: float64(2), int64(2)
memory usage: 3.4 KB
None


,confirmed_cases,deaths,new_confirmed_cases,new_deaths
count,88.000000,88.000000,87.000000,87.000000
mean,2471.238636,85.931818,189.057471,8.413793
std,4276.841374,175.960144,310.491559,15.787507
min,1.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,19.500000,0.000000,5.000000,0.000000
75%,3159.750000,57.000000,347.000000,11.000000
max,16449.000000,732.000000,1474.000000,76.000000


In [26]:
print(df_la)

     confirmed_cases  deaths  new_confirmed_cases  new_deaths
657                1       0                  NaN         NaN
658                1       0                  0.0         0.0
659                1       0                  0.0         0.0
660                1       0                  0.0         0.0
661                1       0                  0.0         0.0
..               ...     ...                  ...         ...
740            12051     577                630.0        76.0
741            12349     601                298.0        24.0
742            13823     619               1474.0        18.0
743            15165     666               1342.0        47.0
744            16449     732               1284.0        66.0

[88 rows x 4 columns]


In [27]:
df_la.dropna(inplace=True)

In [28]:
X = df_la.drop(labels=['confirmed_cases'], axis=1)
print (X)
y = df_la.loc[:,'confirmed_cases']
print (y)
print (y.unique())
X_col_names = X.columns.values
X_col_names

     deaths  new_confirmed_cases  new_deaths
658       0                  0.0         0.0
659       0                  0.0         0.0
660       0                  0.0         0.0
661       0                  0.0         0.0
662       0                  0.0         0.0
..      ...                  ...         ...
740     577                630.0        76.0
741     601                298.0        24.0
742     619               1474.0        18.0
743     666               1342.0        47.0
744     732               1284.0        66.0

[87 rows x 3 columns]
658        1
659        1
660        1
661        1
662        1
       ...  
740    12051
741    12349
742    13823
743    15165
744    16449
Name: confirmed_cases, Length: 87, dtype: int64
[    1     7    11    13    14    19    20    29    32    40    54    69
    94   147   192   231   295   353   411   543   670   814  1230  1482
  1818  2147  2505  3037  3528  4071  4605  5325  5955  6391  6944  7573
  7984  8453  8889  9197  9

array(['deaths', 'new_confirmed_cases', 'new_deaths'], dtype=object)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(X_train)
print(len(X_train),len(y_train))
print(len(X_test),len(y_test))

     deaths  new_confirmed_cases  new_deaths
718      26                252.0         5.0
707       1                 25.0         0.0
691       0                  0.0         0.0
697       0                  2.0         0.0
710       2                 39.0         1.0
681       0                  0.0         0.0
731     225                411.0        25.0
723      66                491.0        12.0
717      21                416.0         8.0
673       0                  0.0         0.0
728     147                436.0        15.0
736     366                595.0        40.0
684       0                  0.0         0.0
743     666               1342.0        47.0
712       4                 58.0         2.0
662       0                  0.0         0.0
704       1                  8.0         0.0
679       0                  0.0         0.0
732     244                469.0        19.0
661       0                  0.0         0.0
719      32                336.0         6.0
688       

In [30]:
print (X_test)
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax 

     deaths  new_confirmed_cases  new_deaths
696       0                  4.0         0.0
702       1                  9.0         1.0
685       0                  0.0         0.0
698       0                  1.0         0.0
694       0                  0.0         0.0
740     577                630.0        76.0
716      13                144.0         3.0
727     132                630.0        13.0
744     732               1284.0        66.0
714       8                132.0         3.0
725      93                534.0        13.0
711       2                 64.0         0.0
706       1                 15.0         0.0
724      80                543.0        14.0
721      44                358.0         7.0
668       0                  0.0         0.0
660       0                  0.0         0.0
693       0                  0.0         0.0
726     119                720.0        26.0
703       1                  3.0         0.0
677       0                  0.0         0.0
739     50

array([[0.00000000e+00, 2.71370421e-03, 0.00000000e+00],
       [1.50150150e-03, 6.10583446e-03, 1.92307692e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.78426052e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.66366366e-01, 4.27408412e-01, 1.46153846e+00],
       [1.95195195e-02, 9.76933514e-02, 5.76923077e-02],
       [1.98198198e-01, 4.27408412e-01, 2.50000000e-01],
       [1.09909910e+00, 8.71099050e-01, 1.26923077e+00],
       [1.20120120e-02, 8.95522388e-02, 5.76923077e-02],
       [1.39639640e-01, 3.62279512e-01, 2.50000000e-01],
       [3.00300300e-03, 4.34192673e-02, 0.00000000e+00],
       [1.50150150e-03, 1.01763908e-02, 0.00000000e+00],
       [1.20120120e-01, 3.68385346e-01, 2.69230769e-01],
       [6.60660661e-02, 2.42876526e-01, 1.34615385e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000

## RBF


In [31]:
svr_rbf = SVR(kernel='rbf', C= 100, gamma = 0.1)

In [32]:
svr_rbf.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [33]:
y_predict = svr_rbf.predict(X_test)
y_predict

array([395.07544576, 639.32215297,   1.09999882,  26.2813348 ,
         1.09999882, 779.22321973, 779.22321973, 779.22321973,
       779.22321973, 774.24451289, 779.22321973, 776.98979912,
       687.95870207, 779.22321973, 779.22321973,   1.09999882,
         1.09999882,   1.09999882, 779.22321973, 334.1844697 ,
         1.09999882, 779.22321973,   1.09999882,   1.09999882,
         1.09999882,  26.2813348 , 779.27852817])

In [34]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

4574.073357873315

# Linear

In [35]:
svr_linear = SVR(kernel='linear', C= 100, gamma = 'auto')

In [36]:
svr_linear.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [37]:
y_predict = svr_linear.predict(X_test)
y_predict

array([8.17497230e+00, 1.63749211e+02, 1.09967645e+00, 2.86850042e+00,
       1.09967645e+00, 1.94140023e+04, 8.38656462e+02, 4.72055408e+03,
       2.14573323e+04, 7.46102685e+02, 3.99438944e+03, 1.42835566e+02,
       4.18976138e+01, 3.95732089e+03, 2.18927587e+03, 1.09967645e+00,
       1.09967645e+00, 1.09967645e+00, 6.41633476e+03, 2.06717262e+01,
       1.09967645e+00, 1.39069954e+04, 1.09967645e+00, 1.09967645e+00,
       1.09967645e+00, 2.86850042e+00, 1.77319185e+03])

In [38]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

1814.5694115942076

# Polynomial


In [39]:
svr_poly = SVR(kernel='poly', C= 100, gamma = 'auto',degree = 2)

In [ ]:
svr_poly.fit(X_train,y_train)

In [ ]:
y_predict = svr_poly.predict(X_test)
y_predict


In [ ]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse